In [1]:
from bs4 import BeautifulSoup
import requests
import time
import os
import pandas as pd

<h1 style="color: green;">get all stocks</h1> 

In [2]:

file_path = r'sp500_companies.csv'
data1 = pd.read_csv(file_path)
print(data1.head())
symbols_array = data1['Symbol'].tolist()

  Exchange Symbol              Shortname               Longname  \
0      NMS   AAPL             Apple Inc.             Apple Inc.   
1      NMS   NVDA     NVIDIA Corporation     NVIDIA Corporation   
2      NMS   MSFT  Microsoft Corporation  Microsoft Corporation   
3      NMS   GOOG          Alphabet Inc.          Alphabet Inc.   
4      NMS  GOOGL          Alphabet Inc.          Alphabet Inc.   

                   Sector                        Industry  Currentprice  \
0              Technology            Consumer Electronics        235.00   
1              Technology                  Semiconductors        138.00   
2              Technology       Software - Infrastructure        418.16   
3  Communication Services  Internet Content & Information        165.05   
4  Communication Services  Internet Content & Information        163.42   

       Marketcap        Ebitda  Revenuegrowth           City State  \
0  3572963475456  1.317810e+11          0.049      Cupertino    CA   
1  338

In [3]:
import re

for symbol in symbols_array:
    sanitized_symbol = re.sub(r"[^a-zA-Z0-9_]", "_", symbol)  # Replace invalid chars with '_'
    sql_query = f"""
        CREATE TABLE [ {sanitized_symbol} ] (
        Id INT IDENTITY(1,1) PRIMARY KEY,
        DateTime DATETIME NOT NULL,
        Title NVARCHAR(MAX) NOT NULL,
        Source NVARCHAR(255) NOT NULL,
        Link NVARCHAR(MAX) NOT NULL);
    """
    with open("create_table.txt", "a", encoding="utf-8") as file:
        file.write(sql_query + "\n")


<h1 style="color: green;">get last scrapped news date</h1>

In [4]:

def get_last_scraped_date_from_file(symbols_array, output_file='latest_scraped_dates.csv'):
    if os.path.exists(output_file):
        existing_df = pd.read_csv(output_file)
    else:
        existing_df = pd.DataFrame(columns=['Stock', 'LatestScrapedDate'])

    stock_dates = {}

    for symbol in symbols_array:
        file_path = f'news_data/{symbol}.csv'

        if not os.path.exists(file_path):
            stock_dates[symbol] = pd.NaT
            continue

        try:
            df = pd.read_csv(file_path, usecols=['datetime'], skipinitialspace=True)
            last_date = df.iloc[-1]['datetime']  # Get the last row's datetime
            stock_dates[symbol] = last_date
        except Exception as e:
            print(f"Error processing {symbol}: {e}")
            stock_dates[symbol] = pd.NaT
    new_df = pd.DataFrame(list(stock_dates.items()), columns=['Stock', 'LatestScrapedDate'])
    final_df = pd.concat([existing_df, new_df]).drop_duplicates(subset=['Stock'], keep='last')
    final_df.to_csv(output_file, index=False)
    return final_df
get_last_scraped_date_from_file(symbols_array)

,Stock,LatestScrapedDate
0,AAPL,2/6/2025 2:40:59 PM
1,NVDA,2/6/2025 3:10:10 PM
2,MSFT,2/6/2025 12:20:20 PM
3,GOOG,2/6/2025 2:27:33 PM
4,GOOGL,2/6/2025 2:27:33 PM
...,...,...
498,MOS,1/28/2025 6:50:48 PM
499,BWA,2/6/2025 1:17:33 PM
500,FMC,2/6/2025 12:09:24 PM
501,PARA,2/6/2025 9:00:07 PM


In [5]:
def get_page(url, retry=0):
    response=requests.get(url)
    html=response.text
    soup= BeautifulSoup(html, 'lxml')
    articles = soup.find_all('div', class_='latest-news__story')
    if len(articles) == 0 and retry < 2:
        retry += 1
        print(f'no articles found, retrying page{page} {retry} time')
        return get_page(url,retry)
    return articles

In [6]:
latestdates = get_last_scraped_date_from_file(symbols_array)
for symbol in symbols_array:
    symbol_latest_date = latestdates.loc[latestdates["Stock"] == symbol, "LatestScrapedDate"].values[0]
    columns = ['datetime', 'title', 'source', 'link'] 
    df =pd.DataFrame(columns=columns)
    counter = 0
    url=f'https://markets.businessinsider.com/news/{symbol.lower()}-stock?miRedirects=1&p={1}'
    response=requests.get(url)
    html=response.text
    soup= BeautifulSoup(html, 'lxml')
    pagination_items = soup.find_all(class_='pagination__item')
    max_page = 0
    for item in pagination_items:
        page_number = item.get('data-pagination-page')
        if page_number:
            max_page = max(max_page, int(page_number))
    print(f'{max_page} pages of news for {symbol} found')
    latest_done = False
    for page in range(1, max_page+1):
        if latest_done:
            break
        url=f'https://markets.businessinsider.com/news/{symbol.lower()}-stock?miRedirects=1&p={page}'
        print(url)
        articles = get_page(url)
        artCount = len(articles)
        for article in articles:
            datetime = article.find('time', class_ ='latest-news__date').get('datetime')
            if datetime == symbol_latest_date:
                latest_done = True
                break
            title=article.find('a', class_ ='news-link').text
            source=article.find('span', class_= 'latest-news__source').text
            link=article.find('a', class_='news-link').get('href')
            df=pd.concat([pd.DataFrame([[datetime, title, source, link]], columns=df.columns), df], ignore_index=True)
            counter += 1
        if not artCount == 0: 
            print(f'page {page} done')
    if counter == 0:
        continue
    print (f'{counter} articles of news for {symbol} scrapped successfully')
    #if the news file does not exist, create it
    if not os.path.exists(f'news_data/{symbol}.csv'):
        df.to_csv(f'news_data/{symbol}.csv', index=False)
        continue
    new_latest_date = df.iloc[-1]['datetime']
    #update latest scraped date in the latest scraped dates file 
    LD_df=pd.read_csv('latest_scraped_dates.csv')
    LD_df.loc[LD_df['Stock']==symbol, 'LatestScrapedDate'] = new_latest_date
    print(f'updating {symbol} latest scraped date to {new_latest_date}')
    file_path = f'news_data/{symbol}.csv'
    existing_df=pd.read_csv(file_path)
    updated_df = pd.concat([existing_df, df],ignore_index=True)
    updated_df.to_csv(file_path,index=False)
    print(existing_df.columns)
    print(df.columns)


420 pages of news for AAPL found
https://markets.businessinsider.com/news/aapl-stock?miRedirects=1&p=1
page 1 done
19 articles of news for AAPL scrapped successfully
updating AAPL latest scraped date to 2/7/2025 10:42:12 PM
Index(['Unnamed: 0', 'datetime', 'title', 'source', 'link'], dtype='object')
Index(['datetime', 'title', 'source', 'link'], dtype='object')
209 pages of news for NVDA found
https://markets.businessinsider.com/news/nvda-stock?miRedirects=1&p=1
page 1 done
18 articles of news for NVDA scrapped successfully
updating NVDA latest scraped date to 2/9/2025 8:34:07 AM
Index(['Unnamed: 0', 'datetime', 'title', 'source', 'link'], dtype='object')
Index(['datetime', 'title', 'source', 'link'], dtype='object')
277 pages of news for MSFT found
https://markets.businessinsider.com/news/msft-stock?miRedirects=1&p=1
page 1 done
20 articles of news for MSFT scrapped successfully
updating MSFT latest scraped date to 2/9/2025 10:45:17 AM
Index(['Unnamed: 0', 'datetime', 'title', 'source

<H1>
automation to get the last news only
</H1>